# OKI Traffic Safety Data Pipeline (Python-first)

## Objective
Build an update-ready spatial dataset from ODOT traffic count station data and public boundary layers.
Outputs include:
- Clean station point layer with county assignment (and optional nearest-road info)
- County-level summary table (station count and AADT stats if available)
- QA report documenting missingness, coordinate validity, and join success

## Why this matters
Transportation planning and safety analysis require reliable spatial datasets. This notebook demonstrates:
- Data ingestion (direct downloads / provided datasets)
- Cleaning and validation (QA)
- Spatial joins and aggregation
- Reproducible outputs suitable for future updates


In [1]:
!pip -q install geopandas pyogrio shapely pandas matplotlib

In [2]:
from pathlib import Path
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# -----------------------
# Project root detection
# -----------------------
ROOT = Path.cwd()

if not (ROOT / "data").exists() and (ROOT / "oki-traffic-safety-arcgis" / "data").exists():
    ROOT = ROOT / "oki-traffic-safety-arcgis"

RAW_DIR = ROOT / "data" / "raw"
PROCESSED_DIR = ROOT / "data" / "processed"
MAPS_DIR = ROOT / "maps"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
MAPS_DIR.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("RAW_DIR exists:", RAW_DIR.exists())
print("Files in RAW_DIR:", len(list(RAW_DIR.glob("*"))))


ROOT: C:\Users\attafuro\Desktop\oki-traffic-safety-arcgis
RAW_DIR exists: True
Files in RAW_DIR: 99


In [3]:
def find_file(patterns, folder=RAW_DIR):
    """Return the first file matching any pattern (case-insensitive)."""
    all_files = list(folder.glob("*"))
    for pat in patterns:
        rx = re.compile(pat, re.IGNORECASE)
        for f in all_files:
            if rx.search(f.name):
                return f
    return None

COUNTIES_SHP = find_file([r"^tl_2025_us_county\.shp$"])
TRAFFIC_FILE = find_file([r"Traffic\s*Count\s*Stations.*\.(csv|txt)$", r"Traffic\s*Count\s*Stations$"])
ROADS_SHP = find_file([r"roads.*\.shp", r"gis_osm_roads.*\.shp"])  # optional

print("COUNTIES_SHP:", COUNTIES_SHP)
print("TRAFFIC_FILE:", TRAFFIC_FILE)
print("ROADS_SHP (optional):", ROADS_SHP)

if COUNTIES_SHP is None:
    raise FileNotFoundError("Could not find tl_2025_us_county.shp in data/raw/")
if TRAFFIC_FILE is None:
    raise FileNotFoundError("Could not find Traffic Count Stations CSV in data/raw/")


COUNTIES_SHP: C:\Users\attafuro\Desktop\oki-traffic-safety-arcgis\data\raw\tl_2025_us_county.shp
TRAFFIC_FILE: C:\Users\attafuro\Desktop\oki-traffic-safety-arcgis\data\raw\Traffic Count Stations.csv
ROADS_SHP (optional): C:\Users\attafuro\Desktop\oki-traffic-safety-arcgis\data\raw\gis_osm_roads_free_1.shp


## Load Census counties and filter to Ohio
We use Census TIGER county boundaries and keep only Ohio (STATEFP = 39).
This reduces dataset size and ensures our analysis scope matches the state.

In [4]:
try:
    ohio_counties = gpd.read_file(COUNTIES_SHP, where="STATEFP='39'", engine="pyogrio")
    print("Loaded Ohio counties via filtered read.")
except Exception as e:
    print("Filtered read not available, reading full file then filtering. Reason:", e)
    counties_all = gpd.read_file(COUNTIES_SHP)
    ohio_counties = counties_all[counties_all["STATEFP"].astype(str) == "39"].copy()

print("Ohio counties:", len(ohio_counties))
print("CRS:", ohio_counties.crs)
ohio_counties.head(3)


Loaded Ohio counties via filtered read.
Ohio counties: 88
CRS: EPSG:4269


,STATEFP,COUNTYFP,COUNTYNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,39,057,01074041,39057,0500000US39057,Greene,Greene County,06,H1,G4020,212,19430,None,A,1071302625,6798109,+39.6874785,-083.8948943,"POLYGON ((-84.10668 39.68891, -84.10662 39.689..."
1,39,045,01074035,39045,0500000US39045,Fairfield,Fairfield County,06,H1,G4020,198,18140,None,A,1306299917,10869534,+39.7476940,-082.6266850,"POLYGON ((-82.49033 39.6617, -82.49042 39.6607..."
2,39,101,01074063,39101,0500000US39101,Marion,Marion County,06,H1,G4020,198,32020,None,A,1045855309,908577,+40.5880337,-083.1688034,"POLYGON ((-83.18826 40.70214, -83.18516 40.702..."


## Load ODOT traffic count stations (CSV) and build point geometry
We:
- read the CSV
- detect latitude/longitude columns
- validate coordinate ranges
- create a GeoDataFrame in EPSG:4326 (WGS84)


In [6]:
import math
import requests
import geopandas as gpd

STATIONS_QUERY_URL = (
    "https://tims.dot.state.oh.us/ags/rest/services/"
    "Roadway_Information/Traffic_Count_Stations/MapServer/0/query"
)

def fetch_arcgis_layer_as_gdf(where="1=1", out_sr=4326, chunk_size=5000, timeout=60):
    """
    Downloads an ArcGIS Server layer using the /query endpoint and returns a GeoDataFrame.

    - where: SQL where clause (default: all records)
    - out_sr: output spatial reference (4326 = lat/lon)
    - chunk_size: pagination size (safe default: 5000)
    """
    # 1) Get total record count
    count_params = {"where": where, "returnCountOnly": "true", "f": "json"}
    count_resp = requests.get(STATIONS_QUERY_URL, params=count_params, timeout=timeout)
    count_resp.raise_for_status()
    total = count_resp.json().get("count")
    if total is None:
        raise ValueError(f"Could not get count. Response: {count_resp.text[:500]}")

    print(f"Total features to download: {total:,}")

    # 2) Download in chunks (pagination)
    all_features = []
    for offset in range(0, total, chunk_size):
        params = {
            "where": where,
            "outFields": "*",
            "returnGeometry": "true",
            "outSR": out_sr,
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": chunk_size,
        }
        resp = requests.get(STATIONS_QUERY_URL, params=params, timeout=timeout)
        resp.raise_for_status()
        gj = resp.json()

        feats = gj.get("features", [])
        all_features.extend(feats)

        done = min(offset + chunk_size, total)
        print(f"Downloaded {done:,}/{total:,}")

        # If the server returns fewer than requested, we might be done early
        if len(feats) == 0:
            break

    gdf = gpd.GeoDataFrame.from_features(all_features, crs=f"EPSG:{out_sr}")
    return gdf

stations = fetch_arcgis_layer_as_gdf(where="1=1", out_sr=4326, chunk_size=5000)
stations.head()


Total features to download: 37,129
Downloaded 5,000/37,129
Downloaded 10,000/37,129
Downloaded 15,000/37,129
Downloaded 20,000/37,129
Downloaded 25,000/37,129
Downloaded 30,000/37,129
Downloaded 35,000/37,129
Downloaded 37,129/37,129


,geometry,STATION_ID_NBR,NLFID,CTL_BEGIN_NBR,COUNTY_CD,COUNT_SOURCE,IS_PERMANENT,ROUTE_TYPE,ROUTE_NBR,AADT_YEAR,AADT,AADT_CARS,AADT_TRUCKS,AADT_SINGLE_TRUCKS,AADT_COMBO_TRUCKS,K_FACTOR,D_FACTOR,TRAFFIC_REPORTS,OBJECTID
0,POINT (-83.61356 38.92753),207290,CADACR00001**C,25.903,ADA,None,No,CR,00001,2024.0,279.0,NaN,NaN,NaN,NaN,0.1051,NaN,http://odot.ms2soft.com/tcds/set_session.asp?l...,1
1,POINT (-83.49154 38.76403),52090,CADACR00001**C,6.347,ADA,None,No,CR,00001,2024.0,105.0,NaN,NaN,NaN,NaN,0.1396,NaN,http://odot.ms2soft.com/tcds/set_session.asp?l...,2
2,POINT (-83.46228 38.72971),52190,CADACR00001**C,1.793,ADA,None,No,CR,00001,2024.0,80.0,NaN,NaN,NaN,NaN,0.1588,NaN,http://odot.ms2soft.com/tcds/set_session.asp?l...,3
3,POINT (-83.46902 38.84273),52290,CADACR00001**C,13.418,ADA,None,No,CR,00001,2024.0,71.0,NaN,NaN,NaN,NaN,0.1376,NaN,http://odot.ms2soft.com/tcds/set_session.asp?l...,4
4,POINT (-83.48949 38.81106),52390,CADACR00001**C,10.511,ADA,None,No,CR,00001,2024.0,261.0,NaN,NaN,NaN,NaN,0.1086,NaN,http://odot.ms2soft.com/tcds/set_session.asp?l...,5


In [7]:
print("Rows:", len(stations))
print("CRS:", stations.crs)
print("Columns:", stations.columns.tolist())

# sanity check geometry bounds (Ohio should be around lon -84..-80, lat 38..42)
print("Bounds:", stations.total_bounds)

# duplicate station ids?
dup = stations["STATION_ID_NBR"].astype(str).duplicated().sum()
print("Duplicate STATION_ID_NBR:", dup)

# missing AADT?
print("Missing AADT (%):", stations["AADT"].isna().mean() * 100)


Rows: 37129
CRS: EPSG:4326
Columns: ['geometry', 'STATION_ID_NBR', 'NLFID', 'CTL_BEGIN_NBR', 'COUNTY_CD', 'COUNT_SOURCE', 'IS_PERMANENT', 'ROUTE_TYPE', 'ROUTE_NBR', 'AADT_YEAR', 'AADT', 'AADT_CARS', 'AADT_TRUCKS', 'AADT_SINGLE_TRUCKS', 'AADT_COMBO_TRUCKS', 'K_FACTOR', 'D_FACTOR', 'TRAFFIC_REPORTS', 'OBJECTID']
Bounds: [-84.81985354  38.4059156  -80.51900486  41.95946364]
Duplicate STATION_ID_NBR: 0
Missing AADT (%): 3.5497858816558487


In [8]:
# Your ohio_counties is EPSG:4269 right now (from TIGER)
# stations is EPSG:4326 (because we requested outSR=4326)

ohio_counties_ll = ohio_counties.to_crs(stations.crs)

# Use 'intersects' to avoid edge-case points on borders missing a match
stations_w_county = gpd.sjoin(
    stations,
    ohio_counties_ll[["GEOID", "NAME", "geometry"]],
    how="left",
    predicate="intersects"
)

stations_w_county[["STATION_ID_NBR", "NAME", "AADT", "AADT_TRUCKS", "IS_PERMANENT"]].head()


,STATION_ID_NBR,NAME,AADT,AADT_TRUCKS,IS_PERMANENT
0,207290,Adams,279.0,NaN,No
1,52090,Adams,105.0,NaN,No
2,52190,Adams,80.0,NaN,No
3,52290,Adams,71.0,NaN,No
4,52390,Adams,261.0,NaN,No


In [9]:
missing = stations_w_county["NAME"].isna().sum()
print("Stations missing county match:", missing)


Stations missing county match: 3
